# Анализ данных эксперимента СФЕРА-2

Анализ данных физического эксперимента СФЕРА проводится с целью выделения событий от широких атмосферных ливней из всех зарегистрированных событий на примере данных 2013 года. 

In [1]:
!date
!pwd

Чт апр 19 17:40:39 MSK 2018
/home/sphere/github/Sphere/SelectEAS/Select


In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mp
import math as mt
import numpy as np

## Чтение данных

Для каждого события были посчитаны значения нескольких параметров. Целевая переменная: EAS.

In [3]:
# чтение данных обработки экспериментальных событий
datum = pd.read_csv('00Events2013.csv', index_col=0)
print('data shape:', datum.shape)
datum.head(5).T

data shape: (3813, 24)


EID,10675,10676,10677,10678,10679
UTC,2013-03-09 18:16:20,2013-03-09 18:16:20,2013-03-09 18:18:12,2013-03-09 18:18:12,2013-03-09 18:18:25
Dt,10,0,30,0,82
Atotal,642,6104408,1339,6205188,7320
TG5time,241,242,239,242,238
TL2time,239,243,238,242,248
TL3time,241,243,242,243,249
TRIGGER,239,242,238,242,238
delay,360,0,112,0,13
H,307.8,307.8,307.8,307.8,307.6
EAS,1,NaN,1,NaN,0


##  Описание параметров

EID - номер события (кадра)

UTC - мировое время UTC

Dt - длительность суммарного импульса по всем каналам события в кадре

Atotal - полная сумма сигналов в кадре, используется для выделения калибровочных кадров

TG5time - время наступления триггера G5

TL2time - время наступления триггера L2

TL3time - время наступления триггера L3

TRIGGER - самое раннее (наименьшее) время наступления триггера из предыдущих трех триггерных времен

delay - время, прошедшее после предыдущего события в секундах

H - высота установки над уровнем наблюдения, !!! не может быть параметром, определяющим тип события !!!

EAS - целевой параметр: 1 - искомый ШАЛ, 0 или Nan - не нужное нам событие. Возможна неточная расстановка признака для не более пяти процентов случаев (около 20 событий) и в сторону несобытие принято за событие. Выставлялся при просмотре глазами. Всего 462 таких события. 

SNRatio_s, SNRatio_n - отношение сигнал/шум в предположениях события(s) или не события(n)

AbsSignal_s, AbsSignal_n -  сигнал

TotalNoise_s, TotalNoise_n - полный шум

NNear3 - три ближайших соседа

Theta - зенитный угол первичной частицы, посчитанный 

dt2_s, dt2_n - временная задержка

ng_s, ng_n -- количество хороших каналов (сигнальных или шумовых)

Crit1, Crit2 - производные критерии:

<pre>
#Crit1 = NGood_s/NGood_n  *  Dt2_n/Dt2_s
data['Crit1'] = (data['ng_s']/data['ng_n'])  *  (data['dt2_n']/data['dt2_s'])
data=data.replace([np.inf, -np.inf], np.nan)
data.fillna(10, inplace=True)

#Crit2 = Noise_n/Noise_s  *  Signal_s/Signal_n
data['Crit2'] = (data['TotalNoise_n']/data['TotalNoise_s'])  *  (data['AbsSignal_s']/data['AbsSignal_n'])
data=data.replace([np.inf, -np.inf], np.nan)
data.fillna(1000, inplace=True)
</pre>

# Отбрасываем заведомые несобытия

In [30]:
# Сначала отбрасываем калибровочные кадры
Amax  = 70000 # максимальная сумма амплитуд в кадре события

print('Raw       :  all:',len(datum), ' EAS:', len(datum[datum.EAS==1]))
cand = datum[:][datum['Atotal'] < Amax] 
print('A < Amax  :  all:',len(cand), ' EAS:', len(cand[cand.EAS==1]))


# Потом отбрасываем события со сбоями в триггере
TrigMin = 220
TrigMax = 260

cand = cand[:][cand['TRIGGER'] > TrigMin]   
print('Trig >', TrigMin, ': all:',len(cand), ' EAS:', len(cand[cand.EAS==1]))

cand = cand[:][cand['TRIGGER'] < TrigMax]  
print('trig <', TrigMax, ': all:',len(cand), ' EAS:', len(cand[cand.EAS==1]))


# Отбрасываем слишком длинные и слишком короткие события
dtmax =     6 # минимальная длительность импульса

cand = cand[:][cand.Dt < 200]  
print('Dt   < 200:  all:',len(cand), ' EAS:', len(cand[cand.EAS==1]))

cand = cand[:][cand.Dt > dtmax]  # Dt>5: 5 eas rejected
print('Dt   > ', dtmax,   ':  all:',len(cand), ' EAS:', len(cand[cand.EAS==1]))

Raw       :  all: 3813  EAS: 462
A < Amax  :  all: 1914  EAS: 462
Trig > 220 : all: 1793  EAS: 462
trig < 260 : all: 1788  EAS: 462
Dt   < 200:  all: 1778  EAS: 462
Dt   >  6 :  all: 1364  EAS: 455


### Сохраним данные в файл

In [32]:
cand.to_csv('01Candidats2013.csv')